In [1]:
#!jupyter nbconvert --to script #1.ipynb

In [2]:
import pandas as pd
import seaborn as sns
import numpy as np

In [3]:
pd.set_option('display.max_rows', 1000)

In [4]:
bene = pd.read_csv("Test_Beneficiarydata-1542969243754.csv")
inpat = pd.read_csv("Test_Inpatientdata-1542969243754.csv")
outpat = pd.read_csv("Test_Outpatientdata-1542969243754.csv")
test = pd.read_csv("Test-1542969243754.csv")

In [ ]:
#Test - bene
print(bene.head(1))
print(bene.shape)
print(bene.dtypes)
print(bene.isnull().sum())

In [ ]:
#Test - in patients
print(inpat.head(1))
print(inpat.shape)
print(inpat.dtypes)
print(inpat.isnull().sum())

In [ ]:
#Test - Outpatients
print(outpat.head(1))
print(outpat.shape)
print(outpat.dtypes)
print(outpat.isnull().sum())

In [ ]:
# Test 
print(test.head(1))
print(test.shape)
print(test.dtypes)
print(test.isnull().sum())

In [9]:
inpat_bene = pd.merge(inpat, bene, left_on = 'BeneID',right_on = 'BeneID', how = 'left')

In [10]:
outpat_bene = pd.merge(outpat, bene, left_on = 'BeneID',right_on = 'BeneID', how = 'left')

In [11]:
testconcat = pd.concat([inpat_bene, outpat_bene])

C:\Users\Deepak\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [12]:
testconcat['DOB'] = pd.to_datetime(testconcat['DOB'])
testconcat['DOD'] = pd.to_datetime(testconcat['DOD'])
testconcat['ClaimEndDt'] = pd.to_datetime(testconcat['ClaimEndDt'])
testconcat['ClaimStartDt'] = pd.to_datetime(testconcat['ClaimStartDt'])
testconcat['AdmissionDt'] = pd.to_datetime(testconcat['AdmissionDt'])
testconcat['DischargeDt'] = pd.to_datetime(testconcat['DischargeDt'])

In [ ]:
#Preprocessing for groupby
testconcat['deceased_sum'] = [1 if pd.isnull(x) else 0 for x in testconcat['DOD']]
testconcat['age'] = ((testconcat['ClaimStartDt'] - testconcat['DOB'])/np.timedelta64(1,'Y')).apply(np.floor)
testconcat['Gender'][testconcat['Gender'] == 2] = 0
testconcat['No_days_admitted'] = (testconcat['DischargeDt']) - (testconcat['AdmissionDt'])
testconcat['No_days_admitted'] = testconcat['No_days_admitted']/np.timedelta64(1,'D')
testconcat['No_days_admitted'][testconcat['No_days_admitted'].isnull()==True]=1
testconcat['No_days_ClaimProcessing'] = (testconcat['ClaimEndDt']-testconcat['ClaimStartDt'])
testconcat['No_days_ClaimProcessing'] = testconcat['No_days_ClaimProcessing']/np.timedelta64(1,'D')
testconcat['RenalDiseaseIndicator'].unique()
testconcat['RenalDiseaseIndicator'][testconcat['RenalDiseaseIndicator'] == 'Y'] = 1

In [14]:
def series_mode(x):
    value = x.mode()
    if len(value)>0:
        return x.mode()[0]
    else:
        raise Exception(value)

In [15]:
def series_median(x):
    return x.median()

In [16]:
def series_sum(x):
    return x.sum()

In [ ]:
#GroupBy

ee1 = testconcat.groupby('Provider').agg({
    "age":[("age_mode",series_mode),("age_median", series_median)],
    "deceased_sum":[("deceased_sum",series_sum)],
    "Gender":[("Gender_mode", series_mode)],
    "Race":[("Race_mode", series_mode)],
    "State":[("State_mode", series_mode)],
    "County":[("County_mode", series_mode)],
    "No_days_admitted":[("No_days_admitted_median", series_median), ("No_days_admitted_mode", series_mode)],
    "No_days_ClaimProcessing":[("No_days_ClaimProcessing_median", series_median), ("No_days_ClaimProcessing_mode", series_mode)],
    "AttendingPhysician":[("AttendingPhysician_mode", series_mode)],
    "RenalDiseaseIndicator":[("RenalDiseaseIndicator_mode",series_mode)],
    "ChronicCond_Alzheimer":[("ChronicCond_Alzheimer_median", series_median), ("ChronicCond_Alzheimer_mode", series_mode)],
    "ChronicCond_Cancer":[("ChronicCond_Cancer_median", series_median), ("ChronicCond_Cancer_mode", series_mode)],
    "ChronicCond_Depression":[("ChronicCond_Depression_median", series_median), ("ChronicCond_Depression_mode", series_mode)],
    "ChronicCond_Diabetes":[("ChronicCond_Diabetes_median", series_median), ("ChronicCond_Diabetes_mode", series_mode)],
    "ChronicCond_Heartfailure":[("ChronicCond_Heartfailure_median", series_median), ("ChronicCond_Heartfailure_mode", series_mode)],
    "ChronicCond_IschemicHeart":[("ChronicCond_IschemicHeart_median", series_median), ("ChronicCond_IschemicHeart_mode", series_mode)],
    "ChronicCond_KidneyDisease":[("ChronicCond_KidneyDisease_median", series_median), ("ChronicCond_KidneyDisease_mode", series_mode)],
    "ChronicCond_ObstrPulmonary":[("ChronicCond_ObstrPulmonary_median", series_median), ("ChronicCond_ObstrPulmonary_mode", series_mode)],
    "ChronicCond_Osteoporasis":[("ChronicCond_Osteoporasis_median", series_median), ("ChronicCond_Osteoporasis_mode", series_mode)],
    "ChronicCond_rheumatoidarthritis":[("ChronicCond_rheumatoidarthritis_median", series_median), ("ChronicCond_rheumatoidarthritis_mode", series_mode)],
    "ChronicCond_stroke":[("ChronicCond_stroke_median", series_median), ("ChronicCond_stroke_mode", series_mode)],
    "NoOfMonths_PartACov":[("NoOfMonths_PartACov_median", series_median), ("NoOfMonths_PartACov_mode", series_mode)],
    "NoOfMonths_PartBCov":[("NoOfMonths_PartBCov_median", series_median), ("NoOfMonths_PartBCov_mode", series_mode)],
    "InscClaimAmtReimbursed":[("InscClaimAmtReimbursed_sum", series_sum), ("InscClaimAmtReimbursed_median", series_median)],
    "OPAnnualDeductibleAmt":[("OPAnnualDeductibleAmt_sum", series_sum), ("OPAnnualDeductibleAmt_median", series_median)],
    "OPAnnualReimbursementAmt":[("OPAnnualReimbursementAmt_sum", series_sum), ("OPAnnualReimbursementAmt_median", series_median)],
    "IPAnnualDeductibleAmt":[("IPAnnualDeductibleAmt_sum", series_sum), ("IPAnnualDeductibleAmt_median", series_median)],
    "IPAnnualReimbursementAmt":[("IPAnnualReimbursementAmt_sum", series_sum), ("IPAnnualReimbursementAmt_median", series_median)],
    "DeductibleAmtPaid":[("DeductibleAmtPaid_sum", series_sum), ("DeductibleAmtPaid_median", series_median)]
})


ee1.columns = ee1.columns.droplevel()

#For OperatingPhysician
OperatingPhysician_mode=pd.DataFrame(testconcat.groupby(['Provider'])["OperatingPhysician"].apply(lambda x: list(x.mode())))
OperatingPhysician_mode['Provider']=OperatingPhysician_mode.index
OperatingPhysician_mode=OperatingPhysician_mode.reset_index(drop=True)
OperatingPhysician_mode.columns=["OperatingPhysician_mode", "Provider"]
OperatingPhysician_mode["Provider"]=OperatingPhysician_mode["Provider"].astype(object)
ee1=ee1.merge(OperatingPhysician_mode, on='Provider')


#For OtherPhysician
OtherPhysician_mode=pd.DataFrame(testconcat.groupby(['Provider'])["OtherPhysician"].apply(lambda x: list(x.mode())))
OtherPhysician_mode['Provider']=OtherPhysician_mode.index
OtherPhysician_mode=OtherPhysician_mode.reset_index(drop=True)
OtherPhysician_mode.columns=["OtherPhysician_mode", "Provider"]
OtherPhysician_mode["Provider"]=OtherPhysician_mode["Provider"].astype(object)
ee1=ee1.merge(OtherPhysician_mode, on='Provider')


#no_of_service_provided
no_of_service_provided = pd.DataFrame(testconcat['Provider'].value_counts())
no_of_service_provided.columns = ['no_of_service_provided']
no_of_service_provided['Provider'] = no_of_service_provided.index
ee1=ee1.merge(no_of_service_provided, on='Provider')





In [18]:
ee1.head().transpose()

,0,1,2,3,4
Provider,PRV51002,PRV51006,PRV51009,PRV51010,PRV51018
age_mode,67,72,70,68,75
age_median,73,73,70,74,74
deceased_sum,202,102,39,38,188
Gender_mode,0,0,0,1,0
Race_mode,1,1,1,1,1
State_mode,1,1,1,1,1
County_mode,10,30,500,260,340
No_days_admitted_median,1,1,1,1,1
No_days_admitted_mode,1,1,1,1,1


In [19]:
test_df = pd.DataFrame({'Provider': list(testconcat['Provider'].unique())})

In [20]:
#ee1.columns = ee1.columns.droplevel()
test_df = test_df.merge(ee1, on = 'Provider')

In [21]:
#train_df = pd.merge(train_df, train_classification, left_on = 'Provider', right_on = 'Provider', how = 'inner')

In [22]:
test_df.head().transpose()

,0,1,2,3,4
Provider,PRV57070,PRV54750,PRV53758,PRV55825,PRV52338
age_mode,65,71,72,76,66
age_median,71,71,74,73,72
deceased_sum,12,38,182,121,1545
Gender_mode,0,0,0,0,0
Race_mode,1,1,1,1,1
State_mode,45,31,22,38,11
County_mode,910,620,20,250,530
No_days_admitted_median,5,1,2,2,1
No_days_admitted_mode,2,1,1,1,1


In [23]:
test_df.to_csv('C:/Users/Deepak/Desktop/SB Capstone - 1/data/Processed_Data/Test.csv')